# Dog vs Cat Classification using CNN

## What is a CNN?
A Convolutional Neural Network (ConvNet/CNN) is a Deep Learning algorithm which can take in an input image, assign importance (learnable weights and biases) to various aspects/objects in the image and be able to differentiate one from the other. 
## What is the role of a CNN?
The role of the ConvNet is to reduce the images into a form which is easier to process, without losing features which are critical for getting a good prediction.
## Difference between a CNN and a normal neural network?
The CNN makes the explicit assumption that the inputs are images, which allows us to encode certain properties into the architecture. These then make the forward function more efficient to implement and vastly reduce the amount of parameters in the network.
## Building a CNN 
In addition to regular layers used in normal neural network stacks a CNN uses additional layers such as Conv2D, MaxPooling2D.

A CNN takes as input tensors of shape (image_height, image_width, image_channels) note this is not including our batch dimension. 

The image height and width are fairly self explanatory=, however **what is the channel of an image?** <br>
The channel is also known as the depth of the image. So for example the channel or depth for an RBG image would be 3 (red, blue and green) a black and white image would have dimension 1, Thr grayness of an image. 

In [ ]:
import os, shutil
import os, shutil
# The path to the directory where the original
# dataset was uncompressed
#original_dataset_dir = r'C:\Users\johnl\Documents\dogs-vs-cats'
# The directory where we will
# store our smaller dataset
#base_dir = r'C:\Users\johnl\Documents\dogs-vs-cats_small'
#os.mkdir(base_dir)
# Directories for our training,
# validation and test splits
#train_dir = os.path.join(base_dir, 'train')


In [ ]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [ ]:
filenames = os.listdir(r'C:\Users\johnl\Documents\dogs-vs-cats\train\train')
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    print(category)
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df.head()
df['category'].value_counts().plot.bar()


In [ ]:

image = load_img(r'C:\Users\johnl\Documents\dogs-vs-cats\train\train\cat.1.jpg')
plt.imshow(image)

# Model

In [ ]:
from keras import layers
from keras import models 
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
 input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
 optimizer=optimizers.RMSprop(lr=1e-4),
 metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
from keras import optimizers
model.compile(loss='binary_crossentropy',
 optimizer='rmsprop',
 metrics=['acc'])

# Training and Validation Generators

In [ ]:
df["category"] = df["category"].replace({0: 'cat', 1: 'dog'}) 

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    width_shift_range=0.15,
    height_shift_range=0.15
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df, "C:\\Users\\johnl\\Documents\\dogs-vs-cats\\train\\train\\" ,
    target_size=(150,150),
    class_mode='binary',
    y_col = 'category',
    batch_size=batch_size
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, "C:\\Users\\johnl\\Documents\\dogs-vs-cats\\train\\train\\" ,
    target_size=(150,150),
    class_mode='binary',
    y_col = 'category',
    batch_size=batch_size
)

In [ ]:
history = model.fit_generator(
 train_generator,
 steps_per_epoch=100,
 epochs=30,
 validation_data=validation_generator,
 validation_steps=50)

# Using Pre-Trained Model

Using VGG16 model
#### Arguments passed to model 
- **weights**: to specify which weight checkpoint to initialize the model from
- **include_top**: meaning do we want to include the densely connected classifer, we dont as we are doing our own binary classification.
- **input_shape**: the shape of the image tensors that we will feed to the network.

In [ ]:
from keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
 include_top=False,
 input_shape=(150, 150, 3))

In [ ]:

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
 optimizer=optimizers.RMSprop(lr=2e-5),
 metrics=['acc'])

In [ ]:
history = model.fit_generator(
 train_generator,
 steps_per_epoch=100,
 epochs=30,
 validation_data=validation_generator,
 validation_steps=50)